In [1]:
import json
import pandas as pd
#import plotly.plotly as py
#import plotly.figure_factory as ff
import vincent

In [2]:
#steps
#get Census files: https://www.census.gov/geo/maps-data/data/tiger-cart-boundary.html
#create geoJSON and topoJSON at: http://mapshaper.org/
#create 1:1 mapping of fips codes to data

#get distribution of lars by county to find best color gradient
#which quantiles make sense to show density well?

In [3]:
#load LAR county level data
lar_county = pd.read_csv("../data/county_counts.txt", sep="|", dtype=object)
lar_county.columns = ['FIPS', 'lars']
lar_county.fillna(-1, inplace=True)
#lar_county[(lar_county.fips==-1)|(lar_county[lar_county.lars==-1])]


In [4]:
#Map the county codes we have in our geometry to those in the
#county_data file, which contains additional rows we don't need
local_path = "/Users/roellk/census_shape_files/counties/"
with open(local_path + 'usa_counties.topo.json', 'r') as f:
    get_id = json.load(f)

In [5]:
#Load topoJSON objects
#with open(local_path+"usa_counties.topo.json") as f:
with open(local_path+"cb_2017_us_county_5m.json") as f:
    county_topo = json.load(f)
with open(local_path + "usa_states.topo.json") as f:
    state_topo = json.load(f)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/roellk/census_shape_files/counties/usa_states.topo.json'

In [6]:
#A little FIPS code type casting to ensure keys match
new_geoms = []
for geom in get_id['objects']['us_counties.geo']['geometries']:
    geom['properties']['FIPS'] = int(geom['properties']['FIPS'])
    new_geoms.append(geom)

In [7]:
get_id['objects']['us_counties.geo']['geometries'] = new_geoms
with open('us_counties.topo.json', 'w') as f:
    json.dump(get_id, f)

In [8]:
#Grab the FIPS codes and load them into a dataframe
geometries = get_id['objects']['us_counties.geo']['geometries']
county_codes = [x['properties']['FIPS'] for x in geometries]
county_df = pd.DataFrame({'FIPS': county_codes}, dtype=str)
county_df = county_df.astype(int)

In [9]:
#Grab the FIPS codes and load them into a dataframe
geometries = get_id['objects']['us_counties.geo']['geometries']
county_codes = [x['properties']['FIPS'] for x in geometries]
county_df = pd.DataFrame({'FIPS': county_codes}, dtype=str)
county_df = county_df.astype(int)

In [10]:
#Read county unemployment data into Dataframe, cast to int for consistency
df = pd.read_csv("../data/county_counts.txt", sep="|", na_values=[' '])
df.columns = ['FIPS', 'lars']
#df = pd.read_csv('data/us_county_data.csv', na_values=[' '])
df['FIPS'] = df['FIPS'].astype(int)

#Perform an inner join, pad NA's with data from nearest county
merged = pd.merge(df, county_df, on='FIPS', how='inner')
merged = merged.fillna(method='pad')

In [11]:
#county_topo = r'/Users/roellk/census_shape_files/counties/usa_counties.topo.json'
geo_data = [{'name': 'counties',
             'url': county_topo,
             'feature': 'us_counties.geo'}]

vis = vincent.Map(data=merged, geo_data=geo_data, scale=1100,
                  projection='albersUsa', data_bind='lars',
                  data_key='FIPS', map_key={'counties': 'properties.FIPS'})
vis.marks[0].properties.enter.stroke_opacity = vincent.ValueRef(value=0.5)
#Change our domain for an even inteager
#vis.scales['color'].domain = [0, 189000]
vis.legend(title='LAR count 2017')
#vis.to_json('vega.json')
vis.scales['color'].type = 'threshold'
vis.rebind(column='lars', brew='YlGnBu')
vis.scales['color'].domain = [0, 150000]
vis.display()

ValueError: url must be one of (str)

In [12]:
vis.marks[0]

In [13]:
import vincent

vincent.core.initialize_notebook()
state_topo = "https://raw.githubusercontent.com/wrobstory/vincent_map_data/master/us_states.topo.json"
county_topo = "https://raw.githubusercontent.com/wrobstory/vincent_map_data/master/us_counties.topo.json"
geo_data = [{'name': 'states',
             'url': state_topo,
             'feature': 'us_states.geo'}]

vis = vincent.Map(geo_data = geo_data, scale = 500, projection = 'albersUsa')
vis.to_json('vega.json')
vis.display()

In [14]:
#this one works
geo_data = [{'name': 'counties',
             'url': county_topo,
             'feature': 'us_counties.geo'},
            {'name': 'states',
             'url': state_topo,
             'feature': 'us_states.geo'}
             ]

vis = vincent.Map(geo_data=geo_data, scale=1000, projection='albersUsa')
#Get rid of State fill, customize stroke color
del vis.marks[1].properties.update
vis.marks[0].properties.update.fill.value = '#084081'
vis.marks[1].properties.enter.stroke.value = '#fff'
vis.marks[0].properties.enter.stroke.value = '#7bccc4'
vis.display()